**딥러닝 3가지 분류 예제**

<hr>

(최종: 쿠팡 해지 예상 고객 예측) — 표준화 없음
<br>

표준화(평균/표준편차로 스케일링) 없이 3가지 분류 문제를 모두 실행 가능하게 구성

**공통 흐름:**

- CSV 준비/로드
- 모델 정의
- 학습
- 평가/예측

**구성:**

- 예제 1: 이진 분류(로지스틱) — 입력 1개(단일 컬럼)
- 예제 2: 다중 클래스 분류(3클래스) — 입력 2개(2컬럼)
- 최종: 쿠팡 해지(Churn) 이진 분류(MLP) — 입력 6개

In [1]:
# =========================
# 0) 환경 준비
# =========================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

np.random.seed(42)
torch.manual_seed(42)

print("torch:", torch.__version__)

torch: 2.9.0+cu126


**예제-1) 이진 분류(입력 1개): 앱 이용점수로 “구독 유지/해지 위험” 분류**
<hr>

📰
**문제**
<br>
1. CSV(app_subscribe.csv)에는 이용점수(engagement_score) 와 구독 유지 여부(subscribe=1/0) 가 있습니다.
2. PyTorch로 가장 단순한 로지스틱 분류(Linear 1→1) 를 학습하고, 점수 35/75의 구독 확률을 예측하세요.

- 입력: engagement_score (1개)
- 라벨: subscribe (0/1)
- 손실: BCEWithLogitsLoss
- 평가지표: Accuracy(0.5 threshold)

In [2]:
# # 1) CSV 생성
# csv_text = """engagement_score,subscribe
# 10,0
# 15,0
# 20,0
# 25,0
# 30,0
# 35,0
# 40,1
# 50,1
# 60,1
# 70,1
# 80,1
# 90,1
# """
# with open("app_subscribe.csv", "w", encoding="utf-8") as f:
#     f.write(csv_text)

In [3]:
df = pd.read_csv("app_subscribe.csv")
X = torch.tensor(df[["engagement_score"]].values, dtype=torch.float32)  # (N,1)
y = torch.tensor(df[["subscribe"]].values, dtype=torch.float32)         # (N,1)

In [4]:
# 모델 선정
model = nn.Linear(1, 1)
# 층을 여러개 쌓았으면 활성화함수 사용해야함.

In [5]:
# 손실함수 선정
loss_fn = nn.BCEWithLogitsLoss() # 이진분류용 손실함수


In [6]:
# 옵티마이저 선정
opt = torch.optim.SGD(model.parameters(), lr=0.001)


In [8]:
# 분류일 때 출력 부분에 활성화함수가 필요함.
# 이진분류 - 시그모이드
# 다중분류 - 소프트맥스

In [7]:
# 훈련시키자.
for epoch in range(6000):
  logits = model(X)
  loss = loss_fn(logits, y)

  opt.zero_grad()
  loss.backward()
  opt.step()

In [17]:
# 예측해보자.
with torch.no_grad():
  # 출력결과를 만들 때 원하는 0,1을 얻어내기 위한 활성화함수가 필요함.
  prob = torch.sigmoid(model(X))
  print(prob)
  pred = (prob >= 0.5).float()
  print(pred)
  acc = (pred == y).float().mean().item()
  print(acc)

tensor([[0.4721],
        [0.5049],
        [0.5376],
        [0.5700],
        [0.6018],
        [0.6328],
        [0.6628],
        [0.7187],
        [0.7686],
        [0.8120],
        [0.8488],
        [0.8795]])
tensor([[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])
0.5833333134651184


In [19]:
# 검증해보자.
print("Train ACC:", acc)

Train ACC: 0.5833333134651184


**예제 2) 다중 클래스 분류(3클래스, 입력 2개): 날씨로 옷차림 추천 분류**
<hr>

**문제** <br>
1. CSV(outfit_weather.csv)에는 기온(temp_c) 과 풍속(wind_mps) 로부터 옷차림 클래스를 예측하는 데이터가 있습니다.

2. 라벨 outfit_class
- 0: 얇게(반팔/얇은 겉옷)
- 1: 보통(긴팔/가벼운 아우터)
- 2: 두껍게(패딩/두꺼운 옷)

<hr>

PyTorch로 3클래스 분류 모델을 학습하고, 아래 입력의 클래스를 예측하세요.


📕 예측 입력값
```
(temp=22, wind=2)
(temp=8, wind=5)
(temp=15, wind=3)
다중 분류는 CrossEntropyLoss를 사용하며, 라벨은 정수(0,1,2) 여야 합니다.

```



In [20]:
# 1) CSV 생성
# csv_text = """temp_c,wind_mps,outfit_class
# 28,1,0
# 26,2,0
# 24,3,0
# 22,1,0
# 21,3,1
# 20,4,1
# 18,2,1
# 17,4,1
# 16,3,1
# 14,2,1
# 13,4,2
# 12,5,2
# 10,3,2
# 9,4,2
# 7,6,2
# """
# with open("outfit_weather.csv", "w", encoding="utf-8") as f:
#     f.write(csv_text)

In [21]:
df = pd.read_csv("outfit_weather.csv")
X = torch.tensor(df[["temp_c", "wind_mps"]].values, dtype=torch.float32)# (N,2)
y = torch.tensor(df["outfit_class"].values, dtype=torch.long)    # (N,) 0/1/2

In [23]:
df.head()

,temp_c,wind_mps,outfit_class
0,28,1,0
1,26,2,0
2,24,3,0
3,22,1,0
4,21,3,1


In [27]:
# 모델 선정
model2 = nn.Linear(2, 3) ## 3은 분류일 때, 클래스개수를 의미함. 다중분류용 손실함수를 사용하는 경우, 클래스별 확률을 가지고 손실을 계산하므로 클래스 개수만큼 넣어주어야함.
## 이진 분류에서 CrossEntropyLoss()를 쓰는 경우 2로 써주어야함.
## 이진 분류용 BCEWithLogitsLoss()를 쓰는 경우 1로 써주어야함.
## 내부적인 계산방식의 차이때문임.
# 층을 여러개 쌓았으면 활성화함수 사용해야함.

In [28]:
# 손실함수 선정
loss_fn = nn.CrossEntropyLoss() # 다중분류용 손실함수

In [31]:
# 옵티마이저 선정
opt = torch.optim.SGD(model2.parameters(), lr=0.05)

In [32]:
# 학습시키고
for epoch in range(1000):
  logits = model2(X)
  loss = loss_fn(logits, y)

  opt.zero_grad()
  loss.backward()
  opt.step()

In [35]:
# 예측하고
with torch.no_grad():
  logits = model2(X)
  print(logits[:2])
  pred = logits.argmax(dim=1)
  print(pred)
  acc = (pred == y).float().mean().item()
  print(acc)

tensor([[ 14.4444,   7.6095, -17.0992],
        [  9.8514,   7.6673, -11.4121]])
tensor([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
0.9333333373069763


In [38]:
## 예측을 하고 싶은데, 온도, 바람을 넣어 옷두께를 예측(판단)해보자.
## 무조건 2차원 텐서로 만들어주어야함.
test = torch.tensor([[4, 2]], dtype=torch.float32)

## 예측해보자.
with torch.no_grad():
  result = model2(test)
  print(result)
  label = torch.argmax(result)
  print(label.item())

tensor([[-4.9775,  2.1756,  6.0866]])
2
